# 3D Plots of charts, vector fields and curves on $\mathbb{S}^2$

This worksheet is based on SageMath 7.5.beta4 and the **three.js** viewer developed at [trac #12402](https://trac.sagemath.org/ticket/12402). 

*NB:* a version of SageMath at least equal to 7.5.beta4 is required:

In [1]:
version()

'SageMath version 7.5.beta4, Release Date: 2016-11-24'

First we set up the notebook to display mathematical objects using LaTeX formatting:

In [2]:
%display latex

We also define a viewer for 3D plots (use `'threejs'` or `'jmol'` for interactive 3D graphics):

In [3]:
viewer3D = 'threejs' # must be 'jmol', 'tachyon', 'threejs' or None (default)

### $\mathbb{S}^2$ as a 2-dimensional differentiable manifold

We declare $\mathbb{S}^2$ as a differentiable manifold of dimension 2 over $\mathbb{R}$ and we endow it with the stereographic charts from the North pole (`stereoN`) and the South pole (`stereoS`):

In [4]:
S2 = Manifold(2, 'S^2', latex_name=r'\mathbb{S}^2', start_index=1)
U = S2.open_subset('U')
V = S2.open_subset('V')
S2.declare_union(U, V)
stereoN.<x,y> = U.chart()
stereoS.<xp,yp> = V.chart(r"xp:x' yp:y'")
stereoN_to_S = stereoN.transition_map(stereoS, (x/(x^2+y^2), y/(x^2+y^2)), 
                                      intersection_name='W',
                                      restrictions1= x^2+y^2!=0, 
                                      restrictions2= xp^2+xp^2!=0)
stereoS_to_N = stereoN_to_S.inverse()
W = U.intersection(V)
eU = stereoN.frame()
eV = stereoS.frame()
S2.frames()

[Coordinate frame (U, (d/dx,d/dy)),
 Coordinate frame (V, (d/dxp,d/dyp)),
 Coordinate frame (W, (d/dx,d/dy)),
 Coordinate frame (W, (d/dxp,d/dyp))]

<h3>Spherical coordinates</h3>
<p>The standard spherical (or polar) coordinates $(\theta,\phi)$ are defined on the open domain $A\subset W \subset \mathbb{S}^2$ that is the complement of the "origin meridian"; since the latter is the half-circle defined by $y=0$ and $x\geq 0$, we declare:</p>

In [5]:
A = W.open_subset('A', coord_def={stereoN.restrict(W): (y!=0, x<0), 
                                  stereoS.restrict(W): (yp!=0, xp<0)})
spher.<th,ph> = A.chart(r'th:(0,pi):\theta ph:(0,2*pi):\phi')
spher_to_stereoN = spher.transition_map(stereoN.restrict(A), 
                                        (sin(th)*cos(ph)/(1-cos(th)),
                                         sin(th)*sin(ph)/(1-cos(th))) )
spher_to_stereoN.set_inverse(2*atan(1/sqrt(x^2+y^2)), atan2(-y,-x)+pi)
spher_to_stereoN.display()

x = -cos(ph)*sin(th)/(cos(th) - 1)
y = -sin(ph)*sin(th)/(cos(th) - 1)

### Embedding of $\mathbb{S}^2$ into $\mathbb{R}^3$

Let us first declare $\mathbb{R}^3$ as a 3-dimensional manifold covered by a single chart (Cartesian coordinates):

In [6]:
R3 = Manifold(3, 'R^3', r'\mathbb{R}^3', start_index=1)
cart.<X,Y,Z> = R3.chart() ; cart

Chart (R^3, (X, Y, Z))

<p>The embedding of the sphere is defined as a differential mapping $\Phi: \mathbb{S}^2 \rightarrow \mathbb{R}^3$:</p>

In [7]:
Phi = S2.diff_map(R3, {(stereoN, cart): [2*x/(1+x^2+y^2), 2*y/(1+x^2+y^2),
                                             (x^2+y^2-1)/(1+x^2+y^2)],
                       (stereoS, cart): [2*xp/(1+xp^2+yp^2), 2*yp/(1+xp^2+yp^2),
                                             (1-xp^2-yp^2)/(1+xp^2+yp^2)]},
                  name='Phi', latex_name=r'\Phi')

In [8]:
Phi.expr(stereoN.restrict(A), cart)
Phi.display()

Phi: S^2 --> R^3
on U: (x, y) |--> (X, Y, Z) = (2*x/(x^2 + y^2 + 1), 2*y/(x^2 + y^2 + 1), (x^2 + y^2 - 1)/(x^2 + y^2 + 1))
on V: (xp, yp) |--> (X, Y, Z) = (2*xp/(xp^2 + yp^2 + 1), 2*yp/(xp^2 + yp^2 + 1), -(xp^2 + yp^2 - 1)/(xp^2 + yp^2 + 1))
on A: (th, ph) |--> (X, Y, Z) = (cos(ph)*sin(th), sin(ph)*sin(th), cos(th))

### 3D plots of coordinate charts on $\mathbb{S}^2$

<p>Let us use $\Phi$ to draw the grid of spherical coordinates $(\theta,\phi)$ in terms of the Cartesian coordinates $(X,Y,Z)$ of $\mathbb{R}^3$:</p>

In [9]:
graph_spher = spher.plot(chart=cart, mapping=Phi, number_values=11, color='blue',
                         label_axes=False)

In [10]:
show(graph_spher, viewer=viewer3D)

Graphics3d Object

We may also use the embedding $\Phi$ to display the stereographic coordinate grid in terms of the Cartesian coordinates in $\mathbb{R}^3$. First for the stereographic coordinates from the North pole:

In [11]:
graph_stereoN = stereoN.plot(chart=cart, mapping=Phi, number_values=25,
                             label_axes=False)
show(graph_stereoN, viewer=viewer3D)

Graphics3d Object

and then have a view with the stereographic coordinates from the South pole superposed (in green):

In [12]:
graph_stereoS = stereoS.plot(chart=cart, mapping=Phi, number_values=25, 
                             color='green', label_axes=False)
show(graph_stereoN + graph_stereoS, viewer=viewer3D)

Graphics3d Object

Let us add the North and South poles to the graphic:

In [13]:
N = V.point((0,0), chart=stereoS, name='N')
S = U.point((0,0), chart=stereoN, name='S')
graph_N = N.plot(chart=cart, mapping=Phi, color='red', label_offset=0.05)
graph_S = S.plot(chart=cart, mapping=Phi, color='green', label_offset=0.05)
show(graph_stereoN + graph_stereoS + graph_N + graph_S, viewer=viewer3D)

Graphics3d Object

### 3D plot of a vector field on $\mathbb{S}^2$

We define the vector field $v$ in terms of its components in the frame `eU` = $(\partial_x, \partial_y)$ associated with stereographic coordinates from the North pole: 

In [14]:
v = S2.vector_field(name='v')
v[eU,:] = [1, -2]
v.display(eU)

v = d/dx - 2 d/dy

In [15]:
v.add_comp_by_continuation(eV, W, chart=stereoS)
v.display(eV)

v = (-xp^2 + 4*xp*yp + yp^2) d/dxp + (-2*xp^2 - 2*xp*yp + 2*yp^2) d/dyp

<p>A 3D view of the vector field $v$ is obtained via the embedding $\Phi$:</p>

In [16]:
graph_v = v.plot(chart=cart, mapping=Phi, chart_domain=spher, number_values=11, 
                 scale=0.2, label_axes=False)
show(graph_v, viewer=viewer3D)

Graphics3d Object

Let us superpose the plot of the spherical coordinate grid:

In [17]:
graph = graph_v + graph_spher
show(graph, viewer=viewer3D, aspect_ratio=1)

Graphics3d Object

Similarly, let us draw the first vector field of the stereographic frame from the North pole, namely $\frac{\partial}{\partial x}$

In [18]:
ex = stereoN.frame()[1]
ex

Vector field d/dx on the Open subset U of the 2-dimensional differentiable manifold S^2

In [19]:
graph_ex = ex.plot(chart=cart, mapping=Phi, chart_domain=spher,
                   number_values=11, scale=0.4, label_axes=False)
show(graph_ex + graph_spher, viewer=viewer3D)

Graphics3d Object

For the second vector field of the stereographic frame from the North pole, namely $\frac{\partial}{\partial y}$, we get:

In [20]:
ey = stereoN.frame()[2]
ey

Vector field d/dy on the Open subset U of the 2-dimensional differentiable manifold S^2

In [21]:
graph_ey = ey.plot(chart=cart, mapping=Phi, chart_domain=spher,
                   number_values=11, scale=0.4, color='red', label_axes=False)
show(graph_ey + graph_spher, viewer=viewer3D)

Graphics3d Object

We may superpose the two graphs, to get a 3D view of the frame associated with the stereographic coordinates from the North pole:

In [22]:
show(graph_spher + graph_ex + graph_ey + sphere(color='lightgrey', opacity=0.4),
     viewer=viewer3D)

Graphics3d Object

### 3D plot of a curve in $\mathbb{S}^2$

In order to define curves in $\mathbb{S}^2$, we first introduce the field of real numbers $\mathbb{R}$ as a 1-dimensional smooth manifold with a canonical coordinate chart:

In [23]:
R.<t> = RealLine() ; print(R)

Real number line R


Let us define a **loxodrome of the sphere** in terms of its parametric equation with respect to the chart `spher` = $(A,(\theta,\phi))$

In [24]:
c = S2.curve({spher: [2*atan(exp(-t/10)), t]}, (t, -oo, +oo), name='c')

Curves in $\mathbb{S}^2$ are considered as morphisms from the manifold $\mathbb{R}$ to the manifold $\mathbb{S}^2$:

In [25]:
c.parent()

Set of Morphisms from Real number line R to 2-dimensional differentiable manifold S^2 in Category of smooth manifolds over Real Field with 53 bits of precision

In [26]:
c.display()

c: R --> S^2
   t |--> (x, y) = (cos(t)*e^(1/10*t), e^(1/10*t)*sin(t))
   t |--> (xp, yp) = (cos(t)*e^(-1/10*t), e^(-1/10*t)*sin(t))
   t |--> (th, ph) = (2*arctan(e^(-1/10*t)), t)

Thanks to the embedding $\Phi$, we may plot $c$ in terms of the Cartesian coordinates of $\mathbb{R}^3$:

In [27]:
graph_c = c.plot(mapping=Phi, max_range=40, plot_points=200, thickness=2,
                 label_axes=False)
show(graph_c + graph_spher, viewer=viewer3D)

Graphics3d Object

The **tangent vector field** (or **velocity vector**) to the curve $c$ is

In [28]:
vc = c.tangent_vector_field()
vc.display()

c' = (1/10*cos(t)*e^(1/10*t) - e^(1/10*t)*sin(t)) d/dx + (cos(t)*e^(1/10*t) + 1/10*e^(1/10*t)*sin(t)) d/dy

Let us plot $c'$ atop $c$:

In [29]:
graph_vc = vc.plot(chart=cart, mapping=Phi, ranges={t: (-20, 20)}, number_values=30, 
                   scale=0.5, color='red', label_axes=False)
show(graph_spher + graph_c + graph_vc , viewer=viewer3D)

Graphics3d Object